In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, TimestampType
from pyspark.sql.functions import col, expr, sum as spark_sum, avg, stddev_pop


In [ ]:
spark = (
SparkSession.builder
.master("local[*]")
.appName("Lab0-OnlineRetail-Warmup")
.config("spark.ui.showConsoleProgress", "false")
.getOrCreate()
)
print("Spark version:", spark.version)
print("Master:", spark.sparkContext.master)


# Define schema for OnlineRetail
online_retail_schema = StructType([
  StructField("InvoiceNo", IntegerType(), True),
  StructField("StockCode", StringType(), True),
  StructField("Description", StringType(), True),
  StructField("Quantity", IntegerType(), True),
  StructField("InvoiceDate", TimestampType(), True),
  StructField("UnitPrice", FloatType(), True),
  StructField("CustomerId", IntegerType(), True),
  StructField("Country", StringType(), True),
])

df = (
spark.read
.option("header", "true")
.option("timestampFormat", "M/d/yyyy H:m")
.schema(online_retail_schema)
.csv("OnlineRetail.csv")
)
print("Rows:", df.count())
df.printSchema()

# Show a few rows
df.show(5, truncate=False)
# Display columns
print("Columns:", df.columns)
# Quick statistical summary on numeric fields
df.describe(["Quantity", "UnitPrice"]).show()

Spark version: 3.5.1
Master: local[*]
Rows: 541909
root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerId|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850     |United Ki

In [ ]:
df.select("Country").show(25, truncate=False)

+--------------+
|Country       |
+--------------+
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
|United Kingdom|
+--------------+
only showing top 25 rows



In [ ]:
df.select("StockCode", "Description", "UnitPrice").show(5, truncate=False)

+---------+-----------------------------------+---------+
|StockCode|Description                        |UnitPrice|
+---------+-----------------------------------+---------+
|85123A   |WHITE HANGING HEART T-LIGHT HOLDER |2.55     |
|71053    |WHITE METAL LANTERN                |3.39     |
|84406B   |CREAM CUPID HEARTS COAT HANGER     |2.75     |
|84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|3.39     |
|84029E   |RED WOOLLY HOTTIE WHITE HEART.     |3.39     |
+---------+-----------------------------------+---------+
only showing top 5 rows



In [ ]:
df.select(df.columns[0:5]).show(5, truncate=False)

+---------+---------+-----------------------------------+--------+-------------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |
+---------+---------+-----------------------------------+--------+-------------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|
+---------+---------+-----------------------------------+--------+-------------------+
only showing top 5 rows



In [ ]:
df_flagged = df.selectExpr(
"*",
"UnitPrice > 100 as HighValueItem"
).select("InvoiceNo", "Description", "UnitPrice", "HighValueItem").show(5, truncate=False)

+---------+-----------------------------------+---------+-------------+
|InvoiceNo|Description                        |UnitPrice|HighValueItem|
+---------+-----------------------------------+---------+-------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |2.55     |false        |
|536365   |WHITE METAL LANTERN                |3.39     |false        |
|536365   |CREAM CUPID HEARTS COAT HANGER     |2.75     |false        |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|3.39     |false        |
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |3.39     |false        |
+---------+-----------------------------------+---------+-------------+
only showing top 5 rows



In [ ]:
df_with_value = df.withColumn("InvoiceValue", col("UnitPrice") * col("Quantity"))

df_line_total = df_with_value.withColumnRenamed("InvoiceValue", "LineTotal").show(5, truncate=True)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+---------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|LineTotal|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+---------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|15.299999|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|    20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|     22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|    20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|    20.34|
+---------+---------+--------------------+--------+-----

In [ ]:
df_agg = df.selectExpr(
"sum(Quantity) as total_quantity",
"sum(UnitPrice * Quantity) as approx_revenue"
)
df_agg.show()

+--------------+----------------+
|total_quantity|  approx_revenue|
+--------------+----------------+
|       5176450|9747747.90762934|
+--------------+----------------+



In [ ]:
df_with_value = df.withColumn("InvoiceValue", col("UnitPrice") * col("Quantity"))
df_with_value.select(
"InvoiceNo", "Description", "UnitPrice", "Quantity", "InvoiceValue"
).show(5, truncate=False)

df_line_total = df_with_value.withColumnRenamed("InvoiceValue", "LineTotal")
df_line_total.select(
"InvoiceNo", "Description", "UnitPrice", "Quantity", "LineTotal"
).show(5, truncate=False)

+---------+-----------------------------------+---------+--------+------------+
|InvoiceNo|Description                        |UnitPrice|Quantity|InvoiceValue|
+---------+-----------------------------------+---------+--------+------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |2.55     |6       |15.299999   |
|536365   |WHITE METAL LANTERN                |3.39     |6       |20.34       |
|536365   |CREAM CUPID HEARTS COAT HANGER     |2.75     |8       |22.0        |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|3.39     |6       |20.34       |
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |3.39     |6       |20.34       |
+---------+-----------------------------------+---------+--------+------------+
only showing top 5 rows

+---------+-----------------------------------+---------+--------+---------+
|InvoiceNo|Description                        |UnitPrice|Quantity|LineTotal|
+---------+-----------------------------------+---------+--------+---------+
|536365   |WHITE HANGING

In [ ]:
df_reduced = df.drop("CustomerId", "StockCode")
print("Original number of columns:", len(df.columns))
print("After drop:", len(df_reduced.columns))
df_reduced.show(5, truncate=False)

Original number of columns: 8
After drop: 6
+---------+-----------------------------------+--------+-------------------+---------+--------------+
|InvoiceNo|Description                        |Quantity|InvoiceDate        |UnitPrice|Country       |
+---------+-----------------------------------+--------+-------------------+---------+--------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |United Kingdom|
|536365   |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |United Kingdom|
|536365   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |United Kingdom|
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |United Kingdom|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |United Kingdom|
+---------+-----------------------------------+--------+-------------------+---------+--------------+
only showing top 5 rows



In [ ]:
avg_qty_per_country = (
df.groupBy("Country")
.agg(avg("Quantity").alias("avg_quantity"))
)

avg_qty_per_country.show(10, truncate=False)

invoice_stats = (
df.groupBy("InvoiceNo")
.agg(
avg("Quantity").alias("avg_quantity"),
stddev_pop("Quantity").alias("std_quantity")
)
)
invoice_stats.show(5, truncate=False)

+------------------+------------------+
|Country           |avg_quantity      |
+------------------+------------------+
|Sweden            |77.13636363636364 |
|Singapore         |22.85589519650655 |
|Germany           |12.369457609268036|
|France            |12.91106696272058 |
|Greece            |10.657534246575343|
|European Community|8.147540983606557 |
|Belgium           |11.18994683421943 |
|Finland           |15.346762589928058|
|Malta             |7.433070866141732 |
|Unspecified       |7.3991031390134525|
+------------------+------------------+
only showing top 10 rows

+---------+------------------+------------------+
|InvoiceNo|avg_quantity      |std_quantity      |
+---------+------------------+------------------+
|536532   |25.36986301369863 |16.850272831671976|
|537632   |1.0               |0.0               |
|538708   |10.61111111111111 |7.150282736359209 |
|538877   |14.258278145695364|27.56989037543246 |
|538993   |9.333333333333334 |2.748737083745107 |
+---------+---

In [ ]:
df_schema = (
spark.read
.option("header", "true")
.option("timestampFormat", "M/d/yyyy H:m")
.schema(online_retail_schema)
.csv("OnlineRetail.csv")
)

print("Schema with explicit schema:")
df_schema.printSchema()


Schema with explicit schema:
root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: float (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:

df_infer = (
spark.read
.option("header", "true")
.option("inferSchema", "true")
.option("timestampFormat", "M/d/yyyy H:m")
.csv("OnlineRetail.csv")
)

print("\nSchema with inferSchema:")
df_infer.printSchema()


Schema with inferSchema:
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
spark.stop()

**Which load was faster (roughly)?**
it seems that the dataframe with the explicitly defined schema was loaded faster than the infered one.

**Why having an explicit schema can be useful in real projects (performance, control, consistency).**
regardless the performance and load time that is reduced by defining the dataframe's schema, it is recommended to define them for a better consistency and control on the code and avoiding the possibilities that the infered type could be non-valid.